In [1]:
# Import modules
import tensorflow as tf
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from urllib.request import urlopen

In [2]:
# Creating a dataframe that stores all the semesters in ascending order from 2010 to 2020
semestersOrder = pd.DataFrame([], 
                              columns=["yearInt", "semInt", "yearString", "semString", "semOrder"], 
                              index=np.linspace(start=1, stop=44, num=44))
semesters = ["VER-1","VER-2","SEM-1","SEM-2"]
row = 1
for year in range(11):
    for sem in range(4):
        semestersOrder.loc[row, "yearInt"] = year + 10
        semestersOrder.loc[row, "semInt"] = sem + 1
        semestersOrder.loc[row, "yearString"] = "20" + str(year+10) + "-20" + str(year+11)
        semestersOrder.loc[row, "semString"] = semesters[sem]
        semestersOrder.loc[row, "semOrder"] = row
        row = row + 1
semestersOrder

,yearInt,semInt,yearString,semString,semOrder
1.0,10,1,2010-2011,VER-1,1
2.0,10,2,2010-2011,VER-2,2
3.0,10,3,2010-2011,SEM-1,3
4.0,10,4,2010-2011,SEM-2,4
5.0,11,1,2011-2012,VER-1,5
6.0,11,2,2011-2012,VER-2,6
7.0,11,3,2011-2012,SEM-1,7
8.0,11,4,2011-2012,SEM-2,8
9.0,12,1,2012-2013,VER-1,9
10.0,12,2,2012-2013,VER-2,10


In [3]:
def addSemOrderCol(df, prefix):
    # Year to int
    df[prefix + "YearInt"] = df[prefix + "Year"].astype(str).str[2:4].astype(int)
    # Semester to int
    df[prefix + "SemesterInt"] = np.select([
    (df[prefix + "Semester"] == "VER-1"), 
    (df[prefix + "Semester"] == "VER-2"), 
    (df[prefix + "Semester"] == "SEM-1"), 
    (df[prefix + "Semester"] == "SEM-2")],
    [1,2,3,4])
    # Semester order
    df["semOrder"] = (df[prefix + "YearInt"] - 10) * 4 + df[prefix + "SemesterInt"]

In [86]:
# Import data from csv files
studentData = pd.read_csv("/home/irixa/git/Capstone/opimi/Sol_3972.csv")
gradesData = pd.read_csv("/home/irixa/git/Capstone/opimi/Sol_3972_Notas.csv")
gpaData = pd.read_csv("/home/irixa/git/Capstone/opimi/GPA.csv")
gradData = pd.read_csv("/home/irixa/git/Capstone/opimi/Grad.csv")

In [150]:
# Student's admission dataframe

# Rename columns to english for conventionality
studentData.rename(columns={
    "Año de Admisión": "yearAdmitted",
    "Programa Académico de Admisión": "programAdmitted",
    "Género": "gender",
    "Escuela Superior": "school",
    "Tipo de Escuela": "schoolType"
}, inplace=True)

# Create new columns with values as integers
studentData["uuid"] = studentData["ID Est"].rank(
    method="dense", ascending=False).astype(int)
studentData["programAdmittedInt"] = studentData["programAdmitted"].rank(
    method="dense", ascending=False).astype(int)
studentData["genderInt"] = np.select([
    (studentData["gender"] == "F"), 
    (studentData["gender"] == "M"), 
    (studentData["gender"] == " ")], 
    [1,2,0]
)
studentData["schoolTypeInt"] = np.select([
    (studentData["schoolType"] == "PUBLICA"), 
    (studentData["schoolType"] == "PRIVADA"), 
    (studentData["schoolType"] == "OTRA")], 
    [1,2,0]
)
studentData["schoolInt"] = studentData["school"].rank(
    method="dense", ascending=False).astype(int)

studentData["yearAdmittedInt"] = studentData.yearAdmitted.astype(str).str[2:4].astype(int)
studentData

,ID Est,yearAdmitted,programAdmitted,gender,IGS,school,schoolType,uuid,programAdmittedInt,genderInt,schoolTypeInt,schoolInt,yearAdmittedInt
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,1,98,12
1,009FA19129,2013,Ingenieria Civil - BC,M,338,SAINT PATRICK'S BILINGUAL SCHO,PRIVADA,22194,22,1,2,31,13
2,009FA512E5,2013,Ingenieria Quimica - BC,F,328,ASUNCION RODRIGUEZ,PUBLICA,22193,17,0,1,321,13
3,009FA559E8,2012,Geologia - BC,F,304,EUGENIO M DE HOSTOS,PUBLICA,22192,27,0,1,162,12
4,009FA63BBC,2018,Ingenieria Computadoras - BC,M,343,ASPIRA,PUBLICA,22191,21,1,1,322,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22190,00DB38A441,2017,Ingenieria Industrial - BC,F,322,LICEO AGUADILLANO,PRIVADA,5,19,0,2,110,17
22191,00DB390BA7,2014,Sistemas Computadorizados de Informacion - BCAE,F,323,COL LA MILAGROSA,PRIVADA,4,3,0,2,272,14
22192,00DB3B2378,2018,Sociologia - BA,M,279,COL SAGRADO CORAZON DE JESUS,PRIVADA,3,2,1,2,251,18
22193,00E93F1845,2013,Quimica - BC,M,319,ESC SUP PRIVADA FUERA DE PR,PRIVADA,2,5,1,2,174,13


In [6]:
# Find correlations between features
studentData.corr()

,yearAdmitted,IGS,uuid,programAdmittedInt,genderInt,schoolTypeInt,schoolInt,yearAdmittedInt
yearAdmitted,1.000000,0.019738,0.001737,0.004673,-0.001699,-0.002349,-0.033185,1.000000
IGS,0.019738,1.000000,0.012072,-0.208325,0.059435,0.070477,0.015390,0.019738
uuid,0.001737,0.012072,1.000000,-0.004962,-0.011987,-0.005269,0.008132,0.001737
programAdmittedInt,0.004673,-0.208325,-0.004962,1.000000,-0.031480,0.000099,0.003087,0.004673
genderInt,-0.001699,0.059435,-0.011987,-0.031480,1.000000,0.028820,-0.013271,-0.001699
schoolTypeInt,-0.002349,0.070477,-0.005269,0.000099,0.028820,1.000000,0.461254,-0.002349
schoolInt,-0.033185,0.015390,0.008132,0.003087,-0.013271,0.461254,1.000000,-0.033185
yearAdmittedInt,1.000000,0.019738,0.001737,0.004673,-0.001699,-0.002349,-0.033185,1.000000


In [143]:
# Student's grades dataframe

# Rename columns
gradesData.rename(columns={
    "Año Académico": "gradeYear",
    "Semestre": "gradeSemester",
    "Curso": "course",
    "Nota": "grade"
}, inplace=True)
gradesData.head()

# Create new columns with values as int
gradesData["courseInt"] = gradesData["course"].rank(
    method="dense", ascending=False).astype(int)
gradesData["gradeInt"] = np.select([
    (gradesData["grade"] == "A"), 
    (gradesData["grade"] == "B"), 
    (gradesData["grade"] == "C"), 
    (gradesData["grade"] == "D"),
    (gradesData["grade"] == "F"),
    (gradesData["grade"] == "W"),
    (gradesData["grade"] == "P")], 
    [1,2,3,4,5,0,0]
)

# Derived columns

# Bool: if student dropped out of the course
gradesData["droppedCourse"] = gradesData["grade"].apply(
    lambda x: True if x == "W" else False)
gradesData["failedCourse"] = gradesData["grade"].apply(
    lambda x: True if x == "F" else False)
# Bool: whether P/F was available that semester or not
gradesData["passOrFailSelected"] = gradesData.grade.apply(lambda x: 1 if x == "P" else 0)
# Bool: whether the course was passed or not
gradesData["passedCourse"] = gradesData.grade.apply(lambda x: 1 if x == "P" else 0)

addSemOrderCol(gradesData, "grade")

In [102]:
# Removing grades before 2010 (grades from 2009 and 1993 were found in data!)
gradesData = gradesData[(gradesData.gradeYearInt >= 10) & (gradesData.gradeYearInt < 93)]

In [92]:
# Find correlation between columns
gradesData

,ID Est,gradeYear,gradeSemester,course,grade,courseInt,gradeInt,droppedCourse,passOrFailSelected,passedCourse,gradeYearInt,gradeSemesterInt,semOrder
0,009FA0F588,2012-2013,SEM-1,CFIT 3005,B,1980,2,False,0,0,12,3,11
1,009FA0F588,2012-2013,SEM-1,ESPA 3101,B,1431,2,False,0,0,12,3,11
2,009FA0F588,2012-2013,SEM-1,BIOL 4015,C,2050,3,False,0,0,12,3,11
3,009FA0F588,2012-2013,SEM-1,ECON 3021,W,1667,0,True,0,0,12,3,11
4,009FA0F588,2012-2013,SEM-1,EDFI 3076,A,1606,1,False,0,0,12,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
793628,00F7BAD7CF,2019-2020,SEM-2,INME 4236,A,604,1,False,0,0,19,4,40
793629,00F7BAD7CF,2019-2020,SEM-2,ININ 4090,A,656,1,False,0,0,19,4,40
793630,00F7BAD7CF,2019-2020,SEM-2,GERE 4085,A,1154,1,False,0,0,19,4,40
793631,00F7BAD7CF,2019-2020,SEM-2,ARTE 4022,A,2111,1,False,0,0,19,4,40


In [103]:
gradesData.gradeInt.unique()
gradesData.loc[(gradesData["semOrder"] > 100)]

,ID Est,gradeYear,gradeSemester,course,grade,courseInt,gradeInt,droppedCourse,passOrFailSelected,passedCourse,gradeYearInt,gradeSemesterInt,semOrder


In [31]:
# Semesters that have grades with P
# df = pd.DataFrame([gradesData.loc[(gradesData["passedCourse"] == 1)].semOrder.unique()]).transpose()
# df.columns=["semOrder"]
# df
# df.join(
#     semestersOrder.set_index("semOrder"), 
#     on="semOrder",
#     how='inner',
#     sort=True)

,semOrder,yearInt,semInt,yearString,semString
4,3,10,3,2010-2011,SEM-1
3,4,10,4,2010-2011,SEM-2
16,5,11,1,2011-2012,VER-1
7,7,11,3,2011-2012,SEM-1
8,11,12,3,2012-2013,SEM-1
19,12,12,4,2012-2013,SEM-2
10,15,13,3,2013-2014,SEM-1
11,16,13,4,2013-2014,SEM-2
5,19,14,3,2014-2015,SEM-1
12,20,14,4,2014-2015,SEM-2


In [10]:
# Why 21297 GPA's are 0? Do these correspond to 1st semester?
gpaData.loc[(gpaData.GPA == 0)]

,ID Est,Año Académico,Semestre,GPA
0,009FA0F588,2012-2013,SEM-1,0.0
10,009FA19129,2013-2014,SEM-1,0.0
30,009FA512E5,2013-2014,SEM-1,0.0
49,009FA559E8,2012-2013,SEM-1,0.0
53,009FA63BBC,2018-2019,SEM-1,0.0
...,...,...,...,...
187097,00DB38A014,2018-2019,SEM-1,0.0
187112,00DB390BA7,2014-2015,SEM-1,0.0
187120,00DB3B2378,2018-2019,SEM-1,0.0
187125,00E93F1845,2013-2014,SEM-1,0.0


In [11]:
# Student's gpa by semester dataframe

# Renaming columns
gpaData.rename(columns={
    "Año Académico": "gpaYear",
    "Semestre": "gpaSemester"
}, inplace=True)

# Create new columns with values as int
addSemOrderCol(gpaData, "gpa")
gpaData

,ID Est,gpaYear,gpaSemester,GPA,gpaYearInt,gpaSemesterInt,semOrder
0,009FA0F588,2012-2013,SEM-1,0.00,12,3,11
1,009FA0F588,2012-2013,SEM-2,3.07,12,4,12
2,009FA0F588,2013-2014,SEM-1,2.53,13,3,15
3,009FA0F588,2013-2014,SEM-2,2.75,13,4,16
4,009FA0F588,2014-2015,SEM-1,2.92,14,3,19
...,...,...,...,...,...,...,...
187146,00F7BAD7CF,2018-2019,SEM-1,3.01,18,3,35
187147,00F7BAD7CF,2018-2019,SEM-2,3.30,18,4,36
187148,00F7BAD7CF,2019-2020,SEM-1,3.31,19,3,39
187149,00F7BAD7CF,2019-2020,SEM-2,3.30,19,4,40


In [174]:
# Student's graduation dataframe

# Renaming columns
gradData.rename(columns={
    "Año Académico": "gradYear",
    "Semestre": "gradSemester",
    "Programa Académico": "gradProgram"
}, inplace=True)

# Create new columns with values as int
addSemOrderCol(gradData, "grad")
gradData

,ID Est,gradYear,gradSemester,gradProgram,gradYearInt,gradSemesterInt,semOrder
0,009FA512E5,2019-2020,SEM-2,Ingenieria Industrial - BC,19,4,40
1,009FACED3F,2017-2018,SEM-2,Ciencia Animal (Industria Pecuaria) - BCA,17,4,32
2,009FAED1A5,2018-2019,SEM-2,Ciencias del Suelo - BCA,18,4,36
3,009FB04764,2015-2016,SEM-2,Sistemas Computadorizados de Informacion - BCAE,15,4,24
4,009FB487ED,2013-2014,SEM-2,Psicologia - BA,13,4,16
...,...,...,...,...,...,...,...
6453,00DB3463BD,2016-2017,SEM-1,Biologia - BC,16,3,27
6454,00DB34AD27,2016-2017,SEM-2,Contabilidad - BCAE,16,4,28
6455,00DB353679,2018-2019,SEM-2,Fisica Teorica - BC,18,4,36
6456,00DB366DBC,2018-2019,SEM-2,Microbiologia Industrial - BC,18,4,36


In [13]:
# Join tables: student admission data and student grades data
studentsGrades = studentData.join(
    gradesData.set_index("ID Est"), 
    on="ID Est",
    how='inner',
    sort=True
)
studentsGrades

,ID Est,yearAdmitted,programAdmitted,gender,IGS,school,schoolType,uuid,programAdmittedInt,genderInt,...,course,grade,courseInt,gradeInt,droppedCourse,passOrFailSelected,passedCourse,gradeYearInt,gradeSemesterInt,semOrder
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,...,CFIT 3005,B,1980,2,False,0,0,12,3,11
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,...,ESPA 3101,B,1431,2,False,0,0,12,3,11
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,...,BIOL 4015,C,2050,3,False,0,0,12,3,11
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,...,ECON 3021,W,1667,0,True,0,0,12,3,11
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,...,EDFI 3076,A,1606,1,False,0,0,12,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22194,00F7BAD7CF,2015,Ingenieria Mecanica - BC,M,351,COL DES INTEGRAL HOSTOS,PRIVADA,1,18,1,...,INME 4236,A,604,1,False,0,0,19,4,40
22194,00F7BAD7CF,2015,Ingenieria Mecanica - BC,M,351,COL DES INTEGRAL HOSTOS,PRIVADA,1,18,1,...,ININ 4090,A,656,1,False,0,0,19,4,40
22194,00F7BAD7CF,2015,Ingenieria Mecanica - BC,M,351,COL DES INTEGRAL HOSTOS,PRIVADA,1,18,1,...,GERE 4085,A,1154,1,False,0,0,19,4,40
22194,00F7BAD7CF,2015,Ingenieria Mecanica - BC,M,351,COL DES INTEGRAL HOSTOS,PRIVADA,1,18,1,...,ARTE 4022,A,2111,1,False,0,0,19,4,40


In [14]:
# List of students that were admitted before 2020 and have no grades
# studentsGrades.loc[(np.isnan(studentsGrades.semester) & (studentsGrades.yearAdmitted != 2020))]

AttributeError: 'DataFrame' object has no attribute 'semester'

In [15]:
# Join dataframes: students admission data and gpa by semester data
studentsGPA = studentData.join(
    gpaData.set_index("ID Est"), 
    on="ID Est",
    how='inner',
    sort=True
)
studentsGPA

,ID Est,yearAdmitted,programAdmitted,gender,IGS,school,schoolType,uuid,programAdmittedInt,genderInt,schoolTypeInt,schoolInt,yearAdmittedInt,gpaYear,gpaSemester,GPA,gpaYearInt,gpaSemesterInt,semOrder
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,1,98,12,2012-2013,SEM-1,0.00,12,3,11
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,1,98,12,2012-2013,SEM-2,3.07,12,4,12
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,1,98,12,2013-2014,SEM-1,2.53,13,3,15
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,1,98,12,2013-2014,SEM-2,2.75,13,4,16
0,009FA0F588,2012,Agricultura General - BCA,F,274,LUIS MUNOZ MARIN,PUBLICA,22195,54,0,1,98,12,2014-2015,SEM-1,2.92,14,3,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22194,00F7BAD7CF,2015,Ingenieria Mecanica - BC,M,351,COL DES INTEGRAL HOSTOS,PRIVADA,1,18,1,2,280,15,2018-2019,SEM-1,3.01,18,3,35
22194,00F7BAD7CF,2015,Ingenieria Mecanica - BC,M,351,COL DES INTEGRAL HOSTOS,PRIVADA,1,18,1,2,280,15,2018-2019,SEM-2,3.30,18,4,36
22194,00F7BAD7CF,2015,Ingenieria Mecanica - BC,M,351,COL DES INTEGRAL HOSTOS,PRIVADA,1,18,1,2,280,15,2019-2020,SEM-1,3.31,19,3,39
22194,00F7BAD7CF,2015,Ingenieria Mecanica - BC,M,351,COL DES INTEGRAL HOSTOS,PRIVADA,1,18,1,2,280,15,2019-2020,SEM-2,3.30,19,4,40


In [16]:
# d1 = studentsGrades.loc[(np.isnan(studentsGrades.semester) & (studentsGrades.yearAdmitted != 2020))]
# d2 = studentsGPA.loc[(np.isnan(studentsGPA.GPA) & (studentsGPA.yearAdmitted != 2020))]
# d3 = d1.join(
#     d2.set_index("ID Est"), 
#     on="ID Est",
#     how="inner",
#     lsuffix="_d1",
#     rsuffix="_d2",
#     sort=True
# )
# d4 = d1.join(
#     d2.set_index("ID Est"), 
#     on="ID Est",
#     how="outer",
#     lsuffix="_d1",
#     rsuffix="_d2",
#     sort=True
# )
# # print(d4)
# # print(d3)
# all = d4.merge(d3.drop_duplicates(), 
#                    how='left', indicator=True)
# all.loc[(all._merge!='both')]

AttributeError: 'DataFrame' object has no attribute 'semester'

In [17]:
# Join tables: student's admissions data with graduation data
studentsGrad = studentData.join(
    gradData.set_index("ID Est"), 
    on="ID Est",
    how='inner',
    sort=True
)
studentsGrad

,ID Est,yearAdmitted,programAdmitted,gender,IGS,school,schoolType,uuid,programAdmittedInt,genderInt,schoolTypeInt,schoolInt,yearAdmittedInt,gradYear,gradSemester,gradProgram,gradYearInt,gradSemesterInt,semOrder
2,009FA512E5,2013,Ingenieria Quimica - BC,F,328,ASUNCION RODRIGUEZ,PUBLICA,22193,17,0,1,321,13,2019-2020,SEM-2,Ingenieria Industrial - BC,19,4,40
17,009FACED3F,2013,Pre-Veterinaria,F,356,COL ESPIRITU SANTO,PRIVADA,22178,8,0,2,276,13,2017-2018,SEM-2,Ciencia Animal (Industria Pecuaria) - BCA,17,4,32
22,009FAED1A5,2014,Ciencia Animal (Industria Pecuaria) - BCA,M,319,CROEM,PUBLICA,22173,47,1,1,211,14,2018-2019,SEM-2,Ciencias del Suelo - BCA,18,4,36
26,009FB04764,2010,Sistemas Computadorizados de Informacion - BCAE,F,295,LUIS MUNOZ MARIN,PUBLICA,22169,3,0,1,98,10,2015-2016,SEM-2,Sistemas Computadorizados de Informacion - BCAE,15,4,24
32,009FB487ED,2010,Psicologia - BA,F,378,LUIS MUNOZ MARIN,PUBLICA,22163,6,0,1,98,10,2013-2014,SEM-2,Psicologia - BA,13,4,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22179,00DB3463BD,2011,Geologia - BC,F,315,LOLA RODZ DE TIO,PUBLICA,16,27,0,1,105,11,2016-2017,SEM-1,Biologia - BC,16,3,27
22180,00DB34AD27,2011,Ingenieria Mecanica - BC,M,349,COL SAGRADA FAMILIA,PRIVADA,15,18,1,2,252,11,2016-2017,SEM-2,Contabilidad - BCAE,16,4,28
22182,00DB353679,2013,Ingenieria Industrial - BC,M,358,ACAD LA INMACULADA,PRIVADA,13,19,1,2,367,13,2018-2019,SEM-2,Fisica Teorica - BC,18,4,36
22184,00DB366DBC,2014,Microbiologia Industrial - BC,M,323,COL SAN AGUSTIN,PRIVADA,11,10,1,2,249,14,2018-2019,SEM-2,Microbiologia Industrial - BC,18,4,36


In [18]:
# Derived columns
studentsGrad["timeToComplete"] = np.select([
    (studentsGrad.gradSemesterInt == 4), 
    (studentsGrad.gradSemesterInt == 3),
    (studentsGrad.gradSemesterInt < 3)], 
    [studentsGrad.gradYearInt - studentsGrad.yearAdmittedInt + 1,
     studentsGrad.gradYearInt - studentsGrad.yearAdmittedInt + 0.5,
     studentsGrad.gradYearInt - studentsGrad.yearAdmittedInt]
)
# something is wrong with the years
# can add (finished?, timeToFinish) columns to student general data
studentsGrad

,ID Est,yearAdmitted,programAdmitted,gender,IGS,school,schoolType,uuid,programAdmittedInt,genderInt,schoolTypeInt,schoolInt,yearAdmittedInt,gradYear,gradSemester,gradProgram,gradYearInt,gradSemesterInt,semOrder,timeToComplete
2,009FA512E5,2013,Ingenieria Quimica - BC,F,328,ASUNCION RODRIGUEZ,PUBLICA,22193,17,0,1,321,13,2019-2020,SEM-2,Ingenieria Industrial - BC,19,4,40,7.0
17,009FACED3F,2013,Pre-Veterinaria,F,356,COL ESPIRITU SANTO,PRIVADA,22178,8,0,2,276,13,2017-2018,SEM-2,Ciencia Animal (Industria Pecuaria) - BCA,17,4,32,5.0
22,009FAED1A5,2014,Ciencia Animal (Industria Pecuaria) - BCA,M,319,CROEM,PUBLICA,22173,47,1,1,211,14,2018-2019,SEM-2,Ciencias del Suelo - BCA,18,4,36,5.0
26,009FB04764,2010,Sistemas Computadorizados de Informacion - BCAE,F,295,LUIS MUNOZ MARIN,PUBLICA,22169,3,0,1,98,10,2015-2016,SEM-2,Sistemas Computadorizados de Informacion - BCAE,15,4,24,6.0
32,009FB487ED,2010,Psicologia - BA,F,378,LUIS MUNOZ MARIN,PUBLICA,22163,6,0,1,98,10,2013-2014,SEM-2,Psicologia - BA,13,4,16,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22179,00DB3463BD,2011,Geologia - BC,F,315,LOLA RODZ DE TIO,PUBLICA,16,27,0,1,105,11,2016-2017,SEM-1,Biologia - BC,16,3,27,5.5
22180,00DB34AD27,2011,Ingenieria Mecanica - BC,M,349,COL SAGRADA FAMILIA,PRIVADA,15,18,1,2,252,11,2016-2017,SEM-2,Contabilidad - BCAE,16,4,28,6.0
22182,00DB353679,2013,Ingenieria Industrial - BC,M,358,ACAD LA INMACULADA,PRIVADA,13,19,1,2,367,13,2018-2019,SEM-2,Fisica Teorica - BC,18,4,36,6.0
22184,00DB366DBC,2014,Microbiologia Industrial - BC,M,323,COL SAN AGUSTIN,PRIVADA,11,10,1,2,249,14,2018-2019,SEM-2,Microbiologia Industrial - BC,18,4,36,5.0


In [19]:
# Import survey data from csv
surveyData = pd.read_csv("/home/irixa/git/Capstone/opimi/Survey.csv")

# Reformating survey data frame

# Inverting columns with rows
surveyData = surveyData.transpose()
# Making first row the header
new_header = surveyData.iloc[0]
surveyData = surveyData[1:]
surveyData.columns = new_header
# Make index serial
surveyData = surveyData.reset_index()
# Change column names
surveyData.columns = [
    "eventName","question","A1","A2","A3","A4","A5","A6","Total" 
]

# Set event names values as string
surveyData.loc[0:3, "eventName"] = "Student Strikes 2017"
surveyData.loc[4:7, "eventName"] = "Hurricane Maria"
surveyData.loc[8:11, "eventName"] = "Power outages 2018-2019"
surveyData.loc[12:15, "eventName"] = "Earthquakes 2019"
surveyData.loc[16:19, "eventName"] = "Covid-19"

# Set question number values as int
surveyData.loc[0:1, "QInt"] = 1
surveyData.loc[4:5, "QInt"] = 1
surveyData.loc[8:9, "QInt"] = 1
surveyData.loc[12:13, "QInt"] = 1
surveyData.loc[16:17, "QInt"] = 1
surveyData.loc[2:3, "QInt"] = 2
surveyData.loc[6:7, "QInt"] = 2
surveyData.loc[10:11, "QInt"] = 2
surveyData.loc[14:15, "QInt"] = 2
surveyData.loc[18:19, "QInt"] = 2

# Set question number as string (Q1 and Q2)
surveyData.question = surveyData.QInt.apply(lambda x: "Q1" if x == 1 else "Q2")

# Set events as int
surveyData["eventInt"] = np.select([
    (surveyData.eventName == "Student Strikes 2017"), 
    (surveyData.eventName == "Hurricane Maria"), 
    (surveyData.eventName == "Power outages 2018-2019"),
    (surveyData.eventName == "Earthquakes 2019"),
    (surveyData.eventName == "Covid-19"),], 
    [1,2,3,4,5]
)
surveyData

,eventName,question,A1,A2,A3,A4,A5,A6,Total,QInt,eventInt
0,Student Strikes 2017,Q1,7,18,16,12,7,32,92,1.0,1
1,Student Strikes 2017,Q1,0.076087,0.195652,0.173913,0.130435,0.076087,0.347826,NaN,1.0,1
2,Student Strikes 2017,Q2,5,20,25,7,2,32,91,2.0,1
3,Student Strikes 2017,Q2,0.0549451,0.21978,0.274725,0.0769231,0.021978,0.351648,NaN,2.0,1
4,Hurricane Maria,Q1,26,25,22,2,7,10,92,1.0,2
5,Hurricane Maria,Q1,0.282609,0.271739,0.23913,0.0217391,0.076087,0.108696,NaN,1.0,2
6,Hurricane Maria,Q2,12,23,23,18,5,10,91,2.0,2
7,Hurricane Maria,Q2,0.131868,0.252747,0.252747,0.197802,0.0549451,0.10989,NaN,2.0,2
8,Power outages 2018-2019,Q1,8,32,17,22,5,8,92,1.0,3
9,Power outages 2018-2019,Q1,0.0869565,0.347826,0.184783,0.23913,0.0543478,0.0869565,NaN,1.0,3


In [20]:
surveyData

,eventName,question,A1,A2,A3,A4,A5,A6,Total,QInt,eventInt
0,Student Strikes 2017,Q1,7,18,16,12,7,32,92,1.0,1
1,Student Strikes 2017,Q1,0.076087,0.195652,0.173913,0.130435,0.076087,0.347826,NaN,1.0,1
2,Student Strikes 2017,Q2,5,20,25,7,2,32,91,2.0,1
3,Student Strikes 2017,Q2,0.0549451,0.21978,0.274725,0.0769231,0.021978,0.351648,NaN,2.0,1
4,Hurricane Maria,Q1,26,25,22,2,7,10,92,1.0,2
5,Hurricane Maria,Q1,0.282609,0.271739,0.23913,0.0217391,0.076087,0.108696,NaN,1.0,2
6,Hurricane Maria,Q2,12,23,23,18,5,10,91,2.0,2
7,Hurricane Maria,Q2,0.131868,0.252747,0.252747,0.197802,0.0549451,0.10989,NaN,2.0,2
8,Power outages 2018-2019,Q1,8,32,17,22,5,8,92,1.0,3
9,Power outages 2018-2019,Q1,0.0869565,0.347826,0.184783,0.23913,0.0543478,0.0869565,NaN,1.0,3


In [191]:
# Dataframe that shows which semesters did each event affect
eventsBySemester = pd.DataFrame(np.array([
    [1, 28], 
    [1, 29], 
    [1, 30], 
    [2, 31], 
    [3, 32], 
    [3, 33], 
    [3, 35], 
    [4, 40], 
    [5, 40], 
    [5, 41], 
    [5, 43]]),
    columns=['eventInt', 'semOrder'])

In [21]:
# Dataframe that shows semesters where Pass or Fail modality was implemented
# passOrFailBySemester = pd.DataFrame(np.array[])

In [192]:
eventsBySemester

,eventInt,semOrder
0,1,28
1,1,29
2,1,30
3,2,31
4,3,32
5,3,33
6,3,35
7,4,40
8,5,40
9,5,41


In [23]:
# surveyData.join(
#     eventsBySemester, 
#     on="eventInt",
#     lsuffix="_survey",
#     rsuffix="_event",
#     how='left',
#     sort=True)

In [24]:
# surveyData.to_csv(r'/home/irixa/git/Capstone/opimi/SurveyFormatted.csv')

In [25]:
surveyData = pd.read_csv('/home/irixa/git/Capstone/opimi/SurveyFormatted.csv')
surveyData

,eventName,Q1A1,Q1A2,Q1A3,Q1A4,Q1A5,Q1A6,Q2A1,Q2A2,Q2A3,Q2A4,Q2A5,Q2A6,eventInt
0,Student Strikes 2017,0.076087,0.130435,0.173913,0.195652,0.076087,0.347826,0.021978,0.076923,0.274725,0.219780,0.054945,0.351648,1
1,Hurricane Maria,0.076087,0.021739,0.239130,0.271739,0.282609,0.108696,0.054945,0.197802,0.252747,0.252747,0.131868,0.109890,2
2,Power outages 2018-2019,0.054348,0.239130,0.184783,0.347826,0.086957,0.086957,0.032967,0.043956,0.560440,0.208791,0.054945,0.098901,3
3,Earthquakes 2019,0.152174,0.195652,0.173913,0.239130,0.130435,0.108696,0.033333,0.022222,0.588889,0.222222,0.055556,0.077778,4
4,Covid-19,0.021978,0.021978,0.098901,0.285714,0.560440,0.010989,0.032967,0.000000,0.098901,0.197802,0.659341,0.010989,5


In [187]:
# Semesters where Pass or Fail modality was implemented
semestersWithPassOrFail = pd.DataFrame([[31],[40],[41]], columns=["semOrder"])
semestersWithPassOrFail

,semOrder
0,31
1,40
2,41


In [193]:
eventsWithSurvey = eventsBySemester.join(
    surveyData.set_index("eventInt"),
    on="eventInt",
    sort=True,
    how="inner",
    lsuffix="_left",
    rsuffix="_right")
# surveyData.append(eventsBySemester, ignore_index=True, sort=False)
eventsWithSurvey

,eventInt,semOrder,eventName,Q1A1,Q1A2,Q1A3,Q1A4,Q1A5,Q1A6,Q2A1,Q2A2,Q2A3,Q2A4,Q2A5,Q2A6
0,1,28,Student Strikes 2017,0.076087,0.130435,0.173913,0.195652,0.076087,0.347826,0.021978,0.076923,0.274725,0.219780,0.054945,0.351648
1,1,29,Student Strikes 2017,0.076087,0.130435,0.173913,0.195652,0.076087,0.347826,0.021978,0.076923,0.274725,0.219780,0.054945,0.351648
2,1,30,Student Strikes 2017,0.076087,0.130435,0.173913,0.195652,0.076087,0.347826,0.021978,0.076923,0.274725,0.219780,0.054945,0.351648
3,2,31,Hurricane Maria,0.076087,0.021739,0.239130,0.271739,0.282609,0.108696,0.054945,0.197802,0.252747,0.252747,0.131868,0.109890
4,3,32,Power outages 2018-2019,0.054348,0.239130,0.184783,0.347826,0.086957,0.086957,0.032967,0.043956,0.560440,0.208791,0.054945,0.098901
5,3,33,Power outages 2018-2019,0.054348,0.239130,0.184783,0.347826,0.086957,0.086957,0.032967,0.043956,0.560440,0.208791,0.054945,0.098901
6,3,35,Power outages 2018-2019,0.054348,0.239130,0.184783,0.347826,0.086957,0.086957,0.032967,0.043956,0.560440,0.208791,0.054945,0.098901
7,4,40,Earthquakes 2019,0.152174,0.195652,0.173913,0.239130,0.130435,0.108696,0.033333,0.022222,0.588889,0.222222,0.055556,0.077778
8,5,40,Covid-19,0.021978,0.021978,0.098901,0.285714,0.560440,0.010989,0.032967,0.000000,0.098901,0.197802,0.659341,0.010989
9,5,41,Covid-19,0.021978,0.021978,0.098901,0.285714,0.560440,0.010989,0.032967,0.000000,0.098901,0.197802,0.659341,0.010989


In [194]:
# Creating dataframe to get statistics by semester
dataBySem = gradesData.groupby(['semOrder']).size().reset_index(name='totalGrades').sort_values(by=["semOrder"])

#### Total drops
droppedCourses = gradesData.groupby(['semOrder', 'droppedCourse']).size().reset_index(name='dropped').sort_values(by=["semOrder"])
droppedCourses = droppedCourses.loc[(droppedCourses.droppedCourse)]
dataBySem = dataBySem.join(
    droppedCourses.set_index("semOrder"),
    on="semOrder",
    sort=True,
    how="inner",
    lsuffix="_left",
    rsuffix="_right",
)
# Drop % in semester = total drops / total grades
dataBySem["totalDropped"] = dataBySem.dropped / dataBySem.totalGrades


#### Total failed
failedCourses = gradesData.groupby(['semOrder', 'failedCourse']).size().reset_index(name='failed').sort_values(by=["semOrder"])
failedCourses = failedCourses.loc[(failedCourses.failedCourse)]
dataBySem = dataBySem.join(
    failedCourses.set_index("semOrder"),
    on="semOrder",
    sort=True,
    how="inner",
    lsuffix="_left",
    rsuffix="_right",
)
# Failed % in semester = total failed / total grades
dataBySem["totalFailed"] = dataBySem.failed / dataBySem.totalGrades


#### Total grad
grad = gradData.groupby(['semOrder']).size().reset_index(name='totalGrad').sort_values(by=["semOrder"])
dataBySem = dataBySem.join(
    grad.set_index("semOrder"),
    on="semOrder",
    sort=True,
    how="outer",
    lsuffix="_left",
    rsuffix="_right",
)
# For some reason we don't have any graduation data before 2013
dataBySem['totalGrad'] = dataBySem['totalGrad'].fillna(0)


#### Deleting helper columns
del dataBySem['droppedCourse']
del dataBySem['dropped']
del dataBySem['failedCourse']
del dataBySem['failed']

dataBySem

,semOrder,totalGrades,totalDropped,totalFailed,totalGrad
0,3,12379,0.068503,0.042734,0.0
1,4,11274,0.061291,0.072202,0.0
2,5,667,0.094453,0.097451,0.0
3,7,19923,0.073031,0.063344,0.0
4,8,18841,0.057375,0.075739,0.0
5,9,1564,0.044118,0.112532,0.0
6,11,27112,0.064990,0.071555,0.0
7,12,25468,0.055403,0.075035,0.0
8,13,2478,0.037127,0.125101,0.0
9,15,34617,0.052026,0.074703,0.0


In [195]:
#### Total admissions
admissions = studentData.groupby(['yearAdmittedInt']).size().reset_index(name='admitted').sort_values(by=['yearAdmittedInt'])
admissions["semOrder"] = (admissions.yearAdmittedInt - 10) * 4 + 3
del admissions['yearAdmittedInt']
admissions
# dataBySem

,admitted,semOrder
0,2069,3
1,1785,7
2,1771,11
3,1855,15
4,2138,19
5,2379,23
6,2374,27
7,2286,31
8,2075,35
9,1794,39


In [196]:
dataBySem = dataBySem.join(
    admissions.set_index("semOrder"),
    on="semOrder",
    sort=True,
    how="outer",
    lsuffix="_left",
    rsuffix="_right",
)
dataBySem['admitted'] = dataBySem['admitted'].fillna(0)
dataBySem

,semOrder,totalGrades,totalDropped,totalFailed,totalGrad,admitted
0.0,3,12379.0,0.068503,0.042734,0.0,2069.0
1.0,4,11274.0,0.061291,0.072202,0.0,0.0
2.0,5,667.0,0.094453,0.097451,0.0,0.0
3.0,7,19923.0,0.073031,0.063344,0.0,1785.0
4.0,8,18841.0,0.057375,0.075739,0.0,0.0
5.0,9,1564.0,0.044118,0.112532,0.0,0.0
6.0,11,27112.0,0.064990,0.071555,0.0,1771.0
7.0,12,25468.0,0.055403,0.075035,0.0,0.0
8.0,13,2478.0,0.037127,0.125101,0.0,0.0
9.0,15,34617.0,0.052026,0.074703,0.0,1855.0


In [201]:
dataBySemWithSurvey = dataBySem.join(
    eventsWithSurvey.set_index("semOrder"),
    on="semOrder",
    sort=True,
    how="outer",
    lsuffix="_left",
    rsuffix="_right",
)
dataBySemWithSurvey['eventInt'] = dataBySemWithSurvey['eventInt'].fillna(0)
dataBySemWithSurvey['eventName'] = dataBySemWithSurvey['eventName'].fillna(0)
dataBySemWithSurvey['Q1A1'] = dataBySemWithSurvey['Q1A1'].fillna(0)
dataBySemWithSurvey['Q1A2'] = dataBySemWithSurvey['Q1A2'].fillna(0)
dataBySemWithSurvey['Q1A3'] = dataBySemWithSurvey['Q1A3'].fillna(0)
dataBySemWithSurvey['Q1A4'] = dataBySemWithSurvey['Q1A4'].fillna(0)
dataBySemWithSurvey['Q1A5'] = dataBySemWithSurvey['Q1A5'].fillna(0)
dataBySemWithSurvey['Q1A6'] = dataBySemWithSurvey['Q1A6'].fillna(1)
dataBySemWithSurvey['Q2A1'] = dataBySemWithSurvey['Q2A1'].fillna(0)
dataBySemWithSurvey['Q2A2'] = dataBySemWithSurvey['Q2A2'].fillna(0)
dataBySemWithSurvey['Q2A3'] = dataBySemWithSurvey['Q2A3'].fillna(0)
dataBySemWithSurvey['Q2A4'] = dataBySemWithSurvey['Q2A4'].fillna(0)
dataBySemWithSurvey['Q2A5'] = dataBySemWithSurvey['Q2A5'].fillna(0)
dataBySemWithSurvey['Q2A6'] = dataBySemWithSurvey['Q2A6'].fillna(1)
dataBySemWithSurvey['totalGrades'] = dataBySemWithSurvey['totalGrades'].fillna(0)
dataBySemWithSurvey['totalDropped'] = dataBySemWithSurvey['totalDropped'].fillna(0)
dataBySemWithSurvey['totalFailed'] = dataBySemWithSurvey['totalFailed'].fillna(0)
dataBySemWithSurvey['totalGrad'] = dataBySemWithSurvey['totalGrad'].fillna(0)

dataBySemWithSurvey

,semOrder,totalGrades,totalDropped,totalFailed,totalGrad,admitted,eventInt,eventName,Q1A1,Q1A2,Q1A3,Q1A4,Q1A5,Q1A6,Q2A1,Q2A2,Q2A3,Q2A4,Q2A5,Q2A6
0.0,3,12379.0,0.068503,0.042734,0.0,2069.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1.0,4,11274.0,0.061291,0.072202,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2.0,5,667.0,0.094453,0.097451,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3.0,7,19923.0,0.073031,0.063344,0.0,1785.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4.0,8,18841.0,0.057375,0.075739,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
5.0,9,1564.0,0.044118,0.112532,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
6.0,11,27112.0,0.064990,0.071555,0.0,1771.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
7.0,12,25468.0,0.055403,0.075035,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
8.0,13,2478.0,0.037127,0.125101,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
9.0,15,34617.0,0.052026,0.074703,0.0,1855.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [213]:
dataBySemWithSurvey = dataBySemWithSurvey.assign(passOrFail=dataBySemWithSurvey.semOrder.isin(semestersWithPassOrFail.semOrder).astype(int))

In [215]:
dataBySemWithSurvey.corr()

,semOrder,totalGrades,totalDropped,totalFailed,totalGrad,admitted,eventInt,Q1A1,Q1A2,Q1A3,Q1A4,Q1A5,Q1A6,Q2A1,Q2A2,Q2A3,Q2A4,Q2A5,Q2A6,passOrFail
semOrder,1.000000,0.289498,-0.740353,-0.349478,0.502246,-0.021664,0.688705,0.505035,0.446053,0.577263,0.664068,0.593025,-0.689281,0.647728,0.282340,0.505099,0.649363,0.539304,-0.690779,0.451869
totalGrades,0.289498,1.000000,0.217434,0.000125,0.741971,0.416223,0.092925,0.204383,0.167720,0.188069,0.145640,-0.002889,-0.132169,0.193188,0.210764,0.223665,0.135467,-0.069475,-0.133578,0.235993
totalDropped,-0.740353,0.217434,1.000000,0.352966,-0.141241,0.326665,-0.564734,-0.310338,-0.292793,-0.308849,-0.446329,-0.499312,0.483699,-0.371619,0.057212,-0.298429,-0.425453,-0.517374,0.486239,-0.265480
totalFailed,-0.349478,0.000125,0.352966,1.000000,-0.071727,-0.205322,-0.550000,-0.313261,-0.146420,-0.333018,-0.425395,-0.586629,0.488490,-0.435970,-0.108611,-0.201408,-0.448735,-0.575363,0.491772,-0.356172
totalGrad,0.502246,0.741971,-0.141241,-0.071727,1.000000,-0.122815,0.360945,0.434248,0.357178,0.363346,0.361402,0.206093,-0.374347,0.367604,0.181951,0.408539,0.366973,0.146915,-0.378260,0.392330
admitted,-0.021664,0.416223,0.326665,-0.205322,-0.122815,1.000000,-0.044740,-0.107810,-0.129207,-0.027308,-0.032211,0.014415,0.043986,0.037605,0.127055,-0.077009,-0.059991,-0.018842,0.045261,-0.031109
eventInt,0.688705,0.092925,-0.564734,-0.550000,0.360945,-0.044740,1.000000,0.572772,0.536824,0.693525,0.890716,0.882659,-0.922389,0.855713,0.245252,0.627316,0.816844,0.829560,-0.924554,0.668951
Q1A1,0.505035,0.204383,-0.310338,-0.313261,0.434248,-0.107810,0.572772,1.000000,0.783892,0.883712,0.735562,0.297128,-0.762727,0.773813,0.658327,0.858271,0.848290,0.148814,-0.768184,0.492877
Q1A2,0.446053,0.167720,-0.292793,-0.146420,0.357178,-0.129207,0.536824,0.783892,1.000000,0.821423,0.789268,0.129023,-0.723960,0.664701,0.443774,0.973835,0.757737,0.032694,-0.721919,0.114495
Q1A3,0.577263,0.188069,-0.308849,-0.333018,0.363346,-0.027308,0.693525,0.883712,0.821423,1.000000,0.921143,0.477866,-0.913543,0.934958,0.793377,0.886360,0.971081,0.329357,-0.912499,0.458826
